<a href="https://colab.research.google.com/github/Tokisaki-Galaxy/PterygiumSeg/blob/master/work1_basemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 导入必要的库
导入PyTorch、OpenCV、Pandas等必要的库，为图像分类模型做准备。

In [ ]:
# ================== 训练参数设置 =================
train_lr=7e-4
train_weight_decay=1e-4
train_model=ResNet34Classifier # type: ignore
num_epochs = 25
log_interval = 10  # 每隔多少个批次打印一次日志

# ================== 数据集路径 =================
# 数据路径
image_dir =          r"f:/train"
# colab路径
colab_zip_path = "/content/drive/My Drive/train.zip"
colab_extract_path = "/content/trains/"
# Kaggle路径
#kaggle_zip_path = "/kaggle/working/train.zip"
#kaggle_extract_path = "/kaggle/working/trains/"
kaggle_extract_path = "/kaggle/input/pterygium/train/"
kaggle_temp_path = "/kaggle/working/"

# =================== 验证集路径 =================
# 验证集路径
val_image_dir =      r"f:/val"
# colab路径
#colab_val_zip_path = "/content/drive/My Drive/val.zip"
#colab_val_extract_path = "/content/val/"
# Kaggle路径
kaggle_val_path = "/kaggle/input/pterygium/val_img/"

# =================== SHAP设置 =================
shap_scaling_factor = 100

In [ ]:
def setup_matplotlib_agg_backend_if_no_gui():
    """
    检查是否可能缺少 GUI 后端（例如，在无头服务器上运行）。
    如果是这种情况，将 Matplotlib 后端设置为 'Agg' 以避免错误。

    应该在首次导入 `matplotlib.pyplot` 之前调用此函数。
    """
    
    # 检查是否在非 Windows 系统上且没有设置 DISPLAY 环境变量
    # 这是判断是否缺少 GUI 的常见启发式方法
    try:
        # 尝试获取 IPython 实例
        shell = get_ipython().__class__.__name__ # type: ignore
        # 'ZMQInteractiveShell' 表示 Jupyter Notebook 或 QtConsole
        # 'TerminalInteractiveShell' 表示 IPython 命令行
        if 'Shell' in shell:
            # Jupyter/IPython 环境
            print('检测到jupyter环境')
            get_ipython().run_line_magic('matplotlib', 'inline') # type: ignore
            return True
        else:
            # 其他情况（理论上不应发生在此 try 块）
            raise NameError
    except NameError:
        print("检测到可能没有 GUI 环境，将 Matplotlib 后端设置为 'Agg'。")
        matplotlib.use('Agg') # type: ignore
        return False      # 标准 Python 解释器 (get_ipython 未定义)
    except Exception as e:
        print(f"警告：尝试将 Matplotlib 后端设置为 'Agg' 时出错: {e}")
        return False

In [ ]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.backends.cudnn as cudnn
from torchvision import transforms, models
import pandas as pd
import os
import zipfile
import shap
import sys
from PIL import Image
import platform
import numpy as np
import glob
from tqdm.autonotebook import tqdm # 好看！
import matplotlib
setup_matplotlib_agg_backend_if_no_gui()
import matplotlib.pyplot as plt
import matplotlib.font_manager

if platform.system() == "Windows":
    num_workers = 0
    print(f"检测到 Windows 系统，将 DataLoader 的 num_workers 设置为 {num_workers}。")
else:
    # 在非 Windows 系统（如 Linux/Colab）上
    num_workers = 4
    print(f"检测到非 Windows 系统 ({platform.system()})，将 DataLoader 的 num_workers 设置为 {num_workers}。")
    # 设置中文字体
    if not os.path.exists('simhei.ttf'):
        os.system(r'wget -O simhei.ttf "https://cdn.jsdelivr.net/gh/Haixing-Hu/latex-chinese-fonts/chinese/%E9%BB%91%E4%BD%93/SimHei.ttf"')
    matplotlib.font_manager.fontManager.addfont('simhei.ttf')
    matplotlib.rc('font', family='SimHei')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 配置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"CUDA 可用: {torch.cuda.is_available()}")
print(f"使用的设备: {device}")

if torch.cuda.is_available():
    cudnn.benchmark = True
    print("cuDNN benchmark 模式已启用")

# 读取和准备数据
从train_classification_label.xlsx读取标签数据，并组织预处理后的图像数据路径。标签包括：0（健康）、1（建议观察）、2（建议手术）。

In [ ]:
# 如果在云端上运行，从 Google Drive 读取数据
if 'google.colab' in sys.modules or os.path.exists("/kaggle/working"):

    if 'google.colab' in sys.modules:
        print('在 Google Colab 环境中运行')
        image_dir = os.path.join(colab_extract_path,"train")
        label_file = os.path.join(image_dir,"train_classification_label.xlsx")
        zip_path = colab_zip_path
        extract_path = colab_extract_path

        # Mount Google Drive
        from google.colab import drive
        drive.mount('/content/drive')
    else:
        print('在 Kaggle 环境中运行')
        # Kaggle 环境下的路径设置
        # image_dir = os.path.join(kaggle_extract_path,"train")
        # label_file = os.path.join(image_dir,"train_classification_label.xlsx")
        # zip_path = kaggle_zip_path
        # extract_path = kaggle_extract_path

        # Google Drive 有每日下载次数限制，可能会导致下载失败
        # if not os.path.exists(zip_path):
        #     from kaggle_secrets import UserSecretsClient
        #     user_secrets = UserSecretsClient()
        #     !gdown --id {user_secrets.get_secret("train_zip_downloadurl")}
        image_dir = os.path.join(kaggle_extract_path,"train")
        label_file = os.path.join(image_dir,"train_classification_label.xlsx")
        val_image_dir = os.path.join(kaggle_val_path,"val_img")

    if not os.path.exists(label_file):
        # 解压数据
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)    
else:
    print(f'不在云端环境中运行,使用本地数据路径{image_dir}')
label_file = os.path.join(image_dir,"train_classification_label.xlsx")

# 自定义数据集类，用于读取图像和标签
class PterygiumDataset(Dataset):
    def __init__(self, label_file, image_dir, transform=None):
        """
        初始化数据集
        :param label_file: 包含图像标签的Excel文件路径
        :param image_dir: 图像文件夹路径
        :param transform: 图像变换操作
        """
        self.labels_df = pd.read_excel(label_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        """
        获取指定索引的图像和标签
        :param idx: 索引
        :return: 图像张量和对应标签
        """
        row = self.labels_df.iloc[idx]
        image_name = row['Image']
        label = row['Pterygium']
        image_folder = f"{int(image_name):04d}"
        image_path = os.path.join(self.image_dir, image_folder, f"{image_folder}.png")

        # 加载图像
        image = Image.open(image_path).convert("RGB")

        # 应用图像变换
        if self.transform:
            image = self.transform(image)

        return image, label

# 数据 Resize
这一步骤是将图像调整为224x224的大小，以适应模型输入要求。
只在Linux运行时使用，因为windows仅用与测试。

In [ ]:
from torchvision.transforms.functional import to_pil_image
target_size = (224, 224) # 目标尺寸
output_format = "PNG" # 输出格式

# --- Transformation Definition ---
# We will perform Resize on GPU. ToTensor conversion happens before moving to GPU.
# Normalization will be done *online* during training dataloading, not offline.
resize_transform = transforms.Resize(target_size, interpolation=transforms.InterpolationMode.BILINEAR, antialias=True)
# BILINEAR is a good default. antialias=True is recommended for downsampling.

# --- Processing Function ---
def resize_and_save_image(img_info, base_input_dir, base_output_dir, transform, device):
    """
    Reads an image, resizes it (potentially on GPU), and saves it.
    """
    try:
        image_name = img_info['Image']
        image_name = f"{int(image_name):04d}"
        if os.path.exists(os.path.join(base_input_dir, f"{image_name}.png")):
            # 验证集图像路径
            input_path = os.path.join(base_input_dir, f"{image_name}.png")
            os.makedirs(base_output_dir, exist_ok=True)
            output_path = os.path.join(base_output_dir, f"{image_name}.{output_format.lower()}")
        else:
            # 训练集图像路径
            input_path = os.path.join(base_input_dir, image_name, f"{image_name}.png")
            # Create corresponding output subdirectory if it doesn't exist
            output_folder_path = os.path.join(base_output_dir, image_name)
            os.makedirs(output_folder_path, exist_ok=True)
            output_path = os.path.join(output_folder_path, f"{image_name}.{output_format.lower()}")

        # 1. Read image using PIL (CPU)
        img_pil = Image.open(input_path).convert("RGB")

        # 2. Convert PIL image to Tensor (CPU, scales to [0, 1])
        img_tensor_cpu = transforms.functional.to_tensor(img_pil) # Output: CxHxW

        # 3. Move tensor to GPU (if available)
        img_tensor_gpu = img_tensor_cpu.to(device)

        # 4. Apply Resize transform (GPU)
        resized_tensor_gpu = transform(img_tensor_gpu)

        # 5. Move resized tensor back to CPU
        resized_tensor_cpu = resized_tensor_gpu.cpu()

        # 6. Convert tensor back to PIL Image (CPU)
        # to_pil_image expects CxHxW tensor in [0, 1] range
        resized_img_pil = to_pil_image(resized_tensor_cpu)

        # 7. Save the resized PIL image (CPU)
        resized_img_pil.save(output_path, format=output_format)
        
        return True # Indicate success

    except FileNotFoundError:
        print(f"错误: 文件未找到 {input_path}")
        return False
    except Exception as e:
        print(f"错误处理图像 {input_path}: {e}")
        return False

if not platform.system() == "Windows":
    if 'google.colab' in sys.modules:
        original_image_dir = os.path.join(colab_extract_path,"train")
        output_dir = os.path.join(colab_extract_path,"train_resized")
    elif os.path.exists("/kaggle/working"):
        original_image_dir = os.path.join(kaggle_extract_path,"train")
        output_dir = os.path.join(kaggle_temp_path,"train_resized")
    else:
        print("错误: 无法识别的环境")
        exit(1)
    image_dir = output_dir

    print(f"输入目录: {original_image_dir}")
    print(f"输出目录: {output_dir}")
    print(f"目标尺寸: {target_size}")

    # Create the main output directory
    os.makedirs(output_dir, exist_ok=True)
    if os.listdir(output_dir):
        print("检测到已存在的resize数据，跳过resize步骤")
    else:
        # Read label file to know which images to process
        try:
            labels_df = pd.read_excel(label_file)
        except FileNotFoundError:
            print(f"错误: 标签文件未找到 {label_file}")
            sys.exit(1) # Exit if label file is missing

        success_count = 0
        error_count = 0

        # Iterate through images listed in the label file
        for index, row in tqdm(labels_df.iterrows(), total=len(labels_df), desc="Resizing Images"):
            if resize_and_save_image(row, original_image_dir, output_dir, resize_transform, device):
                success_count += 1
            else:
                error_count += 1

        print(f"\n处理完成!")
        print(f"成功处理图像数: {success_count}")
        print(f"处理失败图像数: {error_count}")
        print(f"处理后的图像保存在: {output_dir}")

# 创建数据加载器
使用PyTorch的Dataset和DataLoader类创建数据集和加载器，包括数据增强和训练/验证集的划分。

## 模拟高光的数据增强策略

In [ ]:
from PIL import Image, ImageDraw
import random
import torchvision.transforms.functional as TF

class AddRandomHighlight:
    """
    一个 torchvision transform，用于在 PIL 图像上随机添加圆形高光。

    参数:
        p (float): 应用此变换的概率 (0 到 1)。
        max_highlights (int): 单张图像上添加的最大高光数量（实际数量将在1到max_highlights之间随机选择）。
        radius_range (tuple): 一个包含两个整数的元组 (min_radius, max_radius)，指定高光圆形的半径范围。
        color (tuple): 一个包含三个整数的元组 (R, G, B)，指定高光的颜色 (默认为白色)。
    """
    def __init__(self, p=0.5, max_highlights=3, radius_range=(5, 15), color=(255, 255, 255)):
        if not 0.0 <= p <= 1.0:
            raise ValueError(f"概率 p 必须在 [0, 1] 范围内, 但得到 {p}")
        if not (isinstance(max_highlights, int) and max_highlights >= 1):
            raise ValueError(f"最大高光数 max_highlights 必须是 >= 1 的整数, 但得到 {max_highlights}")
        if not (isinstance(radius_range, tuple) and len(radius_range) == 2 and
                isinstance(radius_range[0], int) and isinstance(radius_range[1], int) and
                0 < radius_range[0] <= radius_range[1]):
            raise ValueError(f"半径范围 radius_range 必须是 (min, max) 形式的正整数元组，且 min <= max, 但得到 {radius_range}")
        if not (isinstance(color, tuple) and len(color) == 3 and all(0 <= c <= 255 for c in color)):
            raise ValueError(f"颜色 color 必须是 (R, G, B) 形式的元组，且值在 [0, 255] 范围内, 但得到 {color}")
            
        self.p = p
        self.max_highlights = max_highlights
        self.radius_range = radius_range
        self.color = color

    def __call__(self, img):
        """
        对输入的 PIL 图像应用变换。

        参数:
            img (PIL.Image.Image): 输入的 PIL 图像。

        返回:
            PIL.Image.Image: 可能添加了高光的 PIL 图像。
        """
        # 以概率 p 应用此变换
        if random.random() < self.p:
            # 复制图像以避免修改原始图像（如果原始图像后续还需使用）
            # 如果这是 Compose 链中的一步，通常不需要显式复制
            # img = img.copy() # 如果需要确保不修改原始输入，取消注释此行

            # 随机决定生成多少个高光 (至少1个)
            # 根据用户要求“小于四个”，我们生成 1 到 max_highlights (这里是3) 个
            num_highlights = random.randint(1, self.max_highlights)
            
            # 获取图像尺寸
            width, height = img.size
            
            # 创建 ImageDraw 对象以在图像上绘制
            draw = ImageDraw.Draw(img)

            for _ in range(num_highlights):
                # 随机选择半径
                radius = random.randint(self.radius_range[0], self.radius_range[1])
                
                # 随机选择圆心位置
                # 确保圆心位置加上半径不会超出图像边界太多（允许部分圆在边缘）
                # 稍微限制圆心范围，避免完全生成在图像外的圆心
                center_x = random.randint(0, width - 1) 
                center_y = random.randint(0, height - 1)

                # 计算圆形的边界框 (left, top, right, bottom)
                # ellipse 方法绘制的是指定边界框内的椭圆，如果边界框是正方形，则为圆形
                left = center_x - radius
                top = center_y - radius
                right = center_x + radius
                bottom = center_y + radius
                
                # 绘制实心圆形高光
                draw.ellipse([left, top, right, bottom], fill=self.color)
            
        # 返回处理后的图像（可能是原始图像或添加了高光的图像）
        return img

    def __repr__(self):
        # 提供一个清晰的表示形式，方便调试
        return f"{self.__class__.__name__}(p={self.p}, max_highlights={self.max_highlights}, radius_range={self.radius_range}, color={self.color})"

## 应用数据增强

In [ ]:
# 数据变换
train_transform = transforms.Compose([
    transforms.Resize((256, 256)), # 先放大一点
    transforms.RandomCrop((224, 224)), # 随机裁剪回目标尺寸
    transforms.RandomHorizontalFlip(p=0.5), # 随机水平翻转
    transforms.RandomRotation(degrees=20), # 随机旋转
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1), # 随机颜色抖动
    AddRandomHighlight(p=0.3, max_highlights=3, radius_range=(5, 12)), # 试试添加高光抑制模型关注高光问题
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 定义验证集/测试集的变换
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 划分训练集和验证集，并创建对应的数据加载器
from sklearn.model_selection import train_test_split

# 读取标签文件
labels_df = pd.read_excel(label_file)

# 按照8:2的比例划分训练集和验证集
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=420, stratify=labels_df['Pterygium'])

# 保存划分后的数据集到文件
train_label_file = os.path.join(image_dir, "train_classification_label_train.xlsx")
val_label_file = os.path.join(image_dir, "train_classification_label_val.xlsx")
if os.path.exists("/kaggle/working"):
    train_label_file = os.path.join(kaggle_temp_path, "train_classification_label_train.xlsx")
    val_label_file = os.path.join(kaggle_temp_path, "train_classification_label_val.xlsx")
train_df.to_excel(train_label_file, index=False)
val_df.to_excel(val_label_file, index=False)

# 创建训练集和验证集的数据集对象 (使用不同的 transform)
train_dataset = PterygiumDataset(label_file=train_label_file, image_dir=image_dir, transform=train_transform) # 使用训练变换
val_dataset = PterygiumDataset(label_file=val_label_file, image_dir=image_dir, transform=val_transform) # 使用验证变换

# 创建训练集和验证集的数据加载器
train_loader = DataLoader(train_dataset,
                        batch_size=64,
                        shuffle=True,
                        num_workers=num_workers,
                        prefetch_factor=2 if platform.system() == "Windows" else 10,
                        pin_memory=False if platform.system() == "Windows" else True)
val_loader = DataLoader(val_dataset,
                        batch_size=64,
                        shuffle=False,
                        num_workers=num_workers,
                        prefetch_factor=2 if platform.system() == "Windows" else 10,
                        pin_memory=False if platform.system() == "Windows" else True)

# 构建 ResNet 模型
使用PyTorch的预训练ResNet18模型，修改最后的全连接层以适应3个类别的分类任务。

In [ ]:
# 构建 ResNet18 模型
from torchvision.models import ResNet18_Weights
class ResNet18Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        # 加载预训练的 ResNet18 模型
        self.resnet18 = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        # 替换最后的全连接层以适应3个类别的分类任务
        in_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate), # 添加 Dropout 层
            nn.Linear(in_features, num_classes) # 添加新的全连接层
        )

    def forward(self, x):
        return self.resnet18(x)
    
# 构建 ResNet34 模型
from torchvision.models import ResNet34_Weights
class ResNet34Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        self.resnet34 = models.resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
        in_features = self.resnet34.fc.in_features
        self.resnet34.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate), # 添加 Dropout 层
            nn.Linear(in_features, num_classes) # 添加新的全连接层
        )

    def forward(self, x):
        return self.resnet34(x)
    
# 构建 ResNet50 模型
from torchvision.models import ResNet50_Weights
class ResNet50Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout_rate=0.5):
        super().__init__()
        self.resnet50 = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        in_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate), # 添加 Dropout 层
            nn.Linear(in_features, num_classes) # 添加新的全连接层
        )

    def forward(self, x):
        return self.resnet50(x)
    
# 定义模型
model = train_model(num_classes=3).to(device)

# 将模型移动到 GPU（如果可用）
model = model.to(device)

# 定义带正则化项的损失函数
实现一个包含正则化项的损失函数，使用交叉熵损失作为基础，并添加特定的正则化项来抑制高光问题。

In [ ]:
# 定义损失函数，包含正则化项以抑制高光问题
class HighlightRegularizedLoss(nn.Module):
    def __init__(self, base_loss_fn, lambda_reg=0.01):
        super(HighlightRegularizedLoss, self).__init__()
        self.base_loss_fn = base_loss_fn
        self.lambda_reg = lambda_reg

    def forward(self, outputs, targets, inputs):
        # 基础损失（交叉熵损失）
        base_loss = self.base_loss_fn(outputs, targets)

        # 正则化项：抑制高光问题（假设高光区域的像素值接近1）
        highlight_penalty = torch.mean(torch.clamp(inputs - 0.9, min=0) ** 2)

        # 总损失
        total_loss = base_loss #+ self.lambda_reg * highlight_penalty
        return total_loss

# 定义基础损失函数（交叉熵损失）
base_loss_fn = nn.CrossEntropyLoss()

# 定义包含正则化项的损失函数
criterion = HighlightRegularizedLoss(base_loss_fn=base_loss_fn, lambda_reg=0.01)

# 配置优化器和训练参数

In [ ]:
# 配置优化器和学习率调度器
# 在Adam优化器中添加 weight_decay 参数实现L2正则化
optimizer = optim.AdamW(model.parameters(), lr=train_lr, weight_decay=train_weight_decay)

# 定义学习率调度器，采用余弦退火调度策略
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=25, eta_min=1e-6)

# 训练模型
实现训练循环，包括前向传播、损失计算、反向传播和参数更新，并记录训练过程中的指标。

In [ ]:
# 定义早停类
from copy import deepcopy

class EarlyStopping:
    def __init__(self, patience=7, min_delta=0.0, mode='max'):
        self.patience = patience
        self.min_delta = min_delta # 允许的最小提升量
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.mode = mode
        self.best_model_weights = None

        # 根据模式确定比较操作
        if self.mode == 'min':
            self.delta_sign = -1 # 对于最小值模式，分数需要减少 delta
        else: # mode == 'max'
            self.delta_sign = 1 # 对于最大值模式，分数需要增加 delta

    def __call__(self, val_score, model):
        score = val_score # 直接使用验证分数

        if self.best_score is None:
            # 第一次调用，初始化最佳分数并保存权重
            self.best_score = score
            self.best_model_weights = deepcopy(model.state_dict())
            tqdm.write(f"EarlyStopping: Initialized best score to {self.best_score:.4f}")
        # 检查是否有足够的提升
        elif (score * self.delta_sign) > (self.best_score * self.delta_sign) + self.min_delta:
            # 有足够的提升
            self.best_score = score
            self.best_model_weights = deepcopy(model.state_dict())
            self.counter = 0
            tqdm.write(f"EarlyStopping: Improvement found. Best score updated to {self.best_score:.4f}. Counter reset.")
        else:
            # 没有足够的提升
            self.counter += 1
            tqdm.write(f'EarlyStopping counter: {self.counter} out of {self.patience}. Best score remains {self.best_score:.4f}.')
            if self.counter >= self.patience:
                self.early_stop = True
                tqdm.write("EarlyStopping: Patience reached.")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import time

def train_validate_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, run_identifier="Run"):
    """
    训练并验证模型一个完整的周期，支持早停。

    Args:
        model (nn.Module): 要训练的模型实例。
        train_loader (DataLoader): 训练数据加载器。
        val_loader (DataLoader): 验证数据加载器。
        criterion (nn.Module): 损失函数。
        optimizer (Optimizer): 优化器。
        scheduler (LRScheduler): 学习率调度器。
        num_epochs (int): 最大训练轮数。
        device (torch.device): 计算设备 ('cuda' or 'cpu')。
        run_identifier (str): 用于日志输出的运行标识符。

    Returns:
        tuple: 包含以下元素的元组:
            - float: 最佳验证准确率 (%)。
            - float: 最佳验证Macro F1分数。
            - list: 每个epoch的验证准确率历史。
            - dict: 最佳模型的state_dict。
    """
    start_time_run = time.time()
    print(f"\n--- {run_identifier}: 开始训练 ---")

    # 初始化此运行所需的状态对象
    scaler = torch.amp.GradScaler('cuda')
    early_stopping = EarlyStopping(patience=7, mode='max') # 与之前相同的早停设置
    val_accuracy_history = [] # 存储当前运行的验证准确率历史
    best_model_state_dict = None # 存储最佳模型的状态

    for epoch in range(num_epochs):
        # --- 训练阶段 ---
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        train_loader_tqdm = tqdm(train_loader, desc=f'{run_identifier} Epoch {epoch+1}/{num_epochs}', leave=False)

        for batch_idx, (inputs, targets) in enumerate(train_loader_tqdm):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            with torch.amp.autocast('cuda'):
                outputs = model(inputs)
                # 确保 criterion 也接收 inputs 如果它需要的话 (比如你的 HighlightRegularizedLoss)
                if isinstance(criterion, HighlightRegularizedLoss):
                    loss = criterion(outputs, targets, inputs)
                else:
                    loss = criterion(outputs, targets) # 标准损失函数
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            current_lr = optimizer.param_groups[0]['lr']
            train_loader_tqdm.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100. * correct / total:.2f}%',
                'lr': f'{current_lr:.1e}'
            })

        scheduler.step() # 每个epoch后更新学习率

        # --- 验证阶段 ---
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                with torch.amp.autocast('cuda'):
                    outputs = model(inputs)
                    if isinstance(criterion, HighlightRegularizedLoss):
                        loss = criterion(outputs, targets, inputs)
                    else:
                        loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += targets.size(0)
                val_correct += predicted.eq(targets).sum().item()

        val_accuracy = 100. * val_correct / val_total
        val_accuracy_history.append(val_accuracy)

        tqdm.write(f"{run_identifier} Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
                f"Train Acc: {100. * correct / total:.2f}%, Val Loss: {val_loss / len(val_loader):.4f}, "
                f"Val Acc: {val_accuracy:.2f}%")

        # --- 早停检查 ---
        # 注意: EarlyStopping现在在内部使用tqdm.write
        early_stopping(val_accuracy, model)
        if early_stopping.early_stop:
            tqdm.write(f"{run_identifier}: EarlyStopping triggered at epoch {epoch + 1}.")
            best_model_state_dict = early_stopping.best_model_weights # 获取最佳权重
            break # 提前结束训练

    # 如果训练正常完成（未早停），也要保存最后的最佳权重
    if not early_stopping.early_stop:
        tqdm.write(f"{run_identifier}: Training finished after {num_epochs} epochs.")
        best_model_state_dict = early_stopping.best_model_weights # 获取最后记录的最佳权重

    # --- 使用最佳模型进行最终评估 ---
    if best_model_state_dict:
        model.load_state_dict(best_model_state_dict) # 加载最佳权重
        model.eval()
        final_all_targets = []
        final_all_predictions = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                final_all_targets.extend(targets.cpu().numpy())
                final_all_predictions.extend(predicted.cpu().numpy())

        final_accuracy = accuracy_score(final_all_targets, final_all_predictions) * 100
        final_macro_f1 = f1_score(final_all_targets, final_all_predictions, average='macro')
    else:
        # 如果由于某种原因没有最佳权重（例如训练在第一个epoch就停止且未改进）
        final_accuracy = 0.0
        final_macro_f1 = 0.0
        tqdm.write(f"{run_identifier}: Warning - Could not obtain best model weights.")


    end_time_run = time.time()
    print(f"--- {run_identifier}: 训练完成 ---")
    print(f"最终验证准确率: {final_accuracy:.4f}%")
    print(f"最终验证Macro F1: {final_macro_f1:.4f}")
    print(f"本次运行耗时: {end_time_run - start_time_run:.2f} 秒")

    return final_accuracy, final_macro_f1, val_accuracy_history, best_model_state_dict

In [ ]:
train_validate_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=num_epochs, # 使用之前定义的 num_epochs
        device=device,
        run_identifier=None
    );

# 评估模型性能
在验证集上评估模型性能，计算准确率、混淆矩阵、F1分数等指标，并可视化结果。

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score
import seaborn as sns

# 评估模型性能
model.eval()  # 设置模型为评估模式
all_targets = []
all_predictions = []
misclassified_indices = []
misclassified_info = []

with torch.no_grad():
    for idx, (inputs, targets) in enumerate(val_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        # 前向传播
        outputs = model(inputs)

        # 获取预测结果
        _, predicted = outputs.max(1)

        # 检查错误分类
        misclassified_mask = predicted != targets
        batch_indices = torch.arange(idx * val_loader.batch_size, (idx * val_loader.batch_size) + inputs.size(0))
        batch_misclassified_indices = batch_indices[misclassified_mask.cpu()].tolist()
        misclassified_indices.extend(batch_misclassified_indices)

        # 收集所有目标和预测
        all_targets.extend(targets.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# 计算评估指标
accuracy = accuracy_score(all_targets, all_predictions)
macro_precision = precision_score(all_targets, all_predictions, average='macro')
macro_f1 = f1_score(all_targets, all_predictions, average='macro')
conf_matrix = confusion_matrix(all_targets, all_predictions)

print(f"验证集准确率: {accuracy:.4f}")
print(f"验证集Macro Precision: {macro_precision:.4f}")
print(f"验证集Macro F1分数: {macro_f1:.4f}")

# 可视化混淆矩阵
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["健康", "建议观察", "建议手术"],
            yticklabels=["健康", "建议观察", "建议手术"])
plt.xlabel("预测标签")
plt.ylabel("真实标签")
plt.title("混淆矩阵")
plt.show()

# 获取错误分类图像的文件名和标签信息
if misclassified_indices:
    print(f"\n发现 {len(misclassified_indices)} 个错误分类的样本。")
    # val_df 是之前划分验证集时创建的 DataFrame
    misclassified_samples_df = val_df.iloc[misclassified_indices].copy() 
    misclassified_samples_df['Predicted'] = [all_predictions[i] for i in misclassified_indices]
    misclassified_info = []
    for i, row in misclassified_samples_df.iterrows():
        image_folder = f"{int(row['Image']):04d}"
        # 确定图像路径是来自原始目录还是resize后的目录
        # 注意：这里假设 val_dataset 使用的是 resize 后的 image_dir
        image_path = os.path.join(val_dataset.image_dir, image_folder, f"{image_folder}.png")
        if not os.path.exists(image_path):
            # 如果resize后的路径不存在，尝试原始路径（如果resize在notebook中进行）
            original_val_image_dir = val_image_dir # 使用全局定义的验证集路径
            image_path = os.path.join(original_val_image_dir, f"{image_folder}.png")

        misclassified_info.append({
            'image_path': image_path,
            'true_label': int(row['Pterygium']),
            'predicted_label': int(row['Predicted'])
        })
    print("部分错误分类样本信息:")
    for info in misclassified_info[:5]: # 只显示前5个
        print(f"  路径: {os.path.basename(info['image_path'])}, 真实: {info['true_label']}, 预测: {info['predicted_label']}")
else:
    print("\n验证集上没有发现错误分类的样本。")

# 使用 SHAP 解释部分正确分类的样本

In [ ]:
# --- 找出正确分类的样本 ---
correctly_classified_indices = [i for i, (true, pred) in enumerate(zip(all_targets, all_predictions)) if true == pred]

if len(correctly_classified_indices) >= 3:
    print(f"\n从 {len(correctly_classified_indices)} 个正确分类的样本中随机选择 3 个进行 SHAP 解释...")
    
    # --- 随机选择 3 个 ---
    selected_indices = random.sample(correctly_classified_indices, 3)
    print(f"选择的验证集索引: {selected_indices}")

    # --- 准备选择的样本 ---
    explain_info_correct = []
    explain_images_pil_correct = []
    explain_images_tensor_correct = []

    for index in selected_indices:
        # 从 val_df 获取图像信息
        row = val_df.iloc[index] 
        image_name = row['Image']
        true_label = int(row['Pterygium'])
        
        image_folder = f"{int(image_name):04d}"
        # 确定图像路径 (同之前的逻辑)
        image_path = os.path.join(val_dataset.image_dir, image_folder, f"{image_folder}.png") # 假设用resize后的
        if not os.path.exists(image_path) and 'original_image_dir' in globals(): # 如果resize后的找不到，尝试原始的
            image_path = os.path.join(original_image_dir, image_folder, f"{image_folder}.png") # 需要确保 original_image_dir 已定义
        elif not os.path.exists(image_path): # 如果原始的也找不到，尝试全局验证集路径
            original_val_image_dir = val_image_dir 
            image_path = os.path.join(original_val_image_dir, f"{image_folder}.png")


        explain_info_correct.append({
            'image_path': image_path,
            'true_label': true_label,
            'predicted_label': true_label # 因为是正确分类的样本
        })

        try:
            img = Image.open(image_path).convert("RGB")
            explain_images_pil_correct.append(img) # 保存原始PIL图像
            # 使用验证集变换
            img_tensor = val_transform(img).unsqueeze(0).to(device) 
            explain_images_tensor_correct.append(img_tensor)
        except FileNotFoundError:
            print(f"警告: 找不到图像 {image_path}，跳过解释。")
            # 从 explain_info_correct 中移除对应的条目，以防后续处理出错
            explain_info_correct.pop() 
        except Exception as e:
            print(f"加载或预处理图像 {image_path} 时出错: {e}，跳过解释。")
            explain_info_correct.pop()

    if not explain_images_tensor_correct:
        print("没有可供解释的图像。")
    else:
        # 合并为一个批次
        explain_batch_correct = torch.cat(explain_images_tensor_correct, dim=0)
        num_explain_correct = explain_batch_correct.shape[0] # 实际成功加载的数量
        print(f"成功加载 {num_explain_correct} 个图像进行解释。")

        # --- 准备背景数据 (同之前逻辑) ---
        num_background_samples = 64 # 可以调整数量
        background_indices = np.random.choice(len(train_dataset), num_background_samples, replace=False)
        # 确保获取的是图像张量 train_dataset[i][0]
        background_data = torch.stack([train_dataset[i][0] for i in background_indices]).to(device) 
        print(f"使用 {num_background_samples} 个训练样本作为 SHAP 背景数据。")

        # --- 创建 SHAP 解释器 ---
        # 确保模型处于评估模式
        model.eval() 
        explainer_correct = shap.GradientExplainer(model, background_data)

        # --- 计算 SHAP 值 ---
        print("正在计算正确分类样本的 SHAP 值...")
        shap_values_correct = explainer_correct.shap_values(explain_batch_correct)
        print("SHAP 值计算完成。")

        # --- 准备可视化 ---
        shap_values_np_correct = np.array(shap_values_correct) # (num_classes, num_samples, C, H, W)
        images_np_correct = explain_batch_correct.cpu().numpy().transpose(0, 2, 3, 1) # NCHW -> NHWC

        # 反归一化
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        images_np_denorm_correct = std * images_np_correct + mean
        images_np_denorm_correct = np.clip(images_np_denorm_correct, 0, 1)

        # 准备标签和标题
        class_names=["健康", "建议观察", "建议手术"]
        plot_labels_correct = np.array([class_names] * num_explain_correct) 
        
        sample_titles_correct = [f"样本 (文件: {os.path.basename(info['image_path'])})\n分类成功 真实/预测: {class_names[info['true_label']]}" 
                                for info in explain_info_correct]

        # 调整 SHAP 值格式
        shap_values_plot_correct = np.stack([sv.transpose(0, 2, 3, 1) for sv in shap_values_correct], axis=-1) # N, H, W, C, num_classes

        # --- 可视化前先显示图像和标题 ---
        #print("显示选择的正确分类样本及其标签:")
        #plt.figure(figsize=(5 * num_explain_correct, 5)) # 调整图像大小
        #for i in range(num_explain_correct):
        #    plt.subplot(1, num_explain_correct, i + 1)
        #    plt.imshow(images_np_denorm_correct[i])
        #    plt.title(sample_titles_correct[i], fontsize=10) # 调整字体大小
        #    plt.axis('off')
        #plt.tight_layout() # 调整布局防止标题重叠
        #plt.show()
        
        # --- 可视化 SHAP 结果 ---
        print("生成 SHAP 图像...")
        for i in range(num_explain_correct): # 使用实际成功加载的样本数量
            # 获取第 i 个样本的 SHAP 值 (对所有类别)
            # 确保 SHAP 值列表中的每个元素都是一个包含**单个样本**的批次 (形状为 1, ...)
            shap_values_for_sample_i_list = []
            for sv_per_class in shap_values_correct: # 遍历每个类别的 SHAP 数组
                # 从该类别的 SHAP 数组中提取第 i 个样本，使用切片 [i:i+1] 保留批次维度
                sv_sample_i = sv_per_class[i:i+1] # 形状: (1, C, H, W) 或 (1, H, W, C)

                # (可选) 如果 SHAP 值是 NCHW 格式 (1, C, H, W)，转换为 NHWC (1, H, W, C) 以匹配 image_plot 的期望
                # 进行一个简单的形状检查来判断是否需要转置
                # 假设 C 通常是 3 或 1 (对于灰度图)，H, W 远大于 C
                if sv_sample_i.ndim == 4 and sv_sample_i.shape[1] <= 4 and sv_sample_i.shape[2] > 4: # 粗略判断 NCHW
                    # 从 (1, C, H, W) 转置到 (1, H, W, C)
                    sv_sample_i = sv_sample_i.transpose(0, 2, 3, 1)

                shap_values_for_sample_i_list.append(sv_sample_i)


            # 获取第 i 个样本的图像数据 (保持批次维度为 1)
            image_for_sample_i_batch = images_np_denorm_correct[i:i+1] # 形状: (1, H, W, C)

            # 调用 shap.image_plot 绘制当前样本的解释
            # pixel_values 需要 (N, H, W, C) -> 我们传入 (1, H, W, C)
            # shap_values 需要 list of (N, ...) -> 我们传入 list of (1, H, W, C)
            # labels 需要 list of strings (类别名称) -> 我们传入 class_names
            shap.image_plot(
                [sv * shap_scaling_factor for sv in shap_values_for_sample_i_list], # 对每个类的 SHAP 值应用缩放
                image_for_sample_i_batch,
                labels=class_names, # 使用类别名称作为列标签
                show=False # 不立即显示，以便添加标题
            )

            # 获取当前生成的 matplotlib 图
            fig = plt.gcf()
            # 在图的顶部添加自定义的总体标题
            fig.suptitle(sample_titles_correct[i], fontsize=12, y=1.02) # y 调整标题位置

            # 现在显示带有标题的图
            plt.show()

elif len(correctly_classified_indices) > 0:
    print(f"\n正确分类的样本数量 ({len(correctly_classified_indices)}) 不足 3 个，无法选择 3 个进行解释。")
else:
    print("\n验证集中没有正确分类的样本可供解释。")

# 使用 SHAP 解释错误分类的样本

In [ ]:
if misclassified_info:
    print("\n--- 使用 SHAP 解释部分错误分类样本 ---")
    
    # 1. 选择要解释的样本 (选择前 N 个)
    num_explain = min(10, len(misclassified_info))
    explain_info = misclassified_info[:num_explain]
    explain_images_pil = []
    explain_images_tensor = []

    print(f"准备解释 {num_explain} 个样本...")
    for info in explain_info:
        try:
            img = Image.open(info['image_path']).convert("RGB")
            explain_images_pil.append(img) # 保存原始PIL图像用于绘图标题
            # 使用验证集的变换进行预处理，并移动到设备
            img_tensor = val_transform(img).unsqueeze(0).to(device)
            explain_images_tensor.append(img_tensor)
        except FileNotFoundError:
            print(f"警告: 找不到图像 {info['image_path']}，跳过解释。")
        except Exception as e:
            print(f"加载或预处理图像 {info['image_path']} 时出错: {e}，跳过解释。")

    if not explain_images_tensor:
        print("没有可供解释的图像。")
    else:
        # 合并为一个批次
        explain_batch = torch.cat(explain_images_tensor, dim=0)

        # 2. 准备背景数据 (使用训练集的一小部分)
        num_background_samples = 64
        background_indices = np.random.choice(len(train_dataset), num_background_samples, replace=False)
        background_data = torch.stack([train_dataset[i][0] for i in background_indices]).to(device) 

        print(f"使用 {num_background_samples} 个训练样本作为 SHAP 背景数据。")

        # 3. 创建 SHAP 解释器
        # 对于 PyTorch 模型，GradientExplainer 或 DeepExplainer 是常用选择
        # GradientExplainer 通常更通用
        explainer = shap.GradientExplainer(model, background_data)

        # 4. 计算 SHAP 值
        print("正在计算 SHAP 值...")

        shap_values = explainer.shap_values(explain_batch)
        print("SHAP 值计算完成。")

        # --- 检查 SHAP 值 ---
        shap_values_np = np.array(shap_values) # 转换为 NumPy 数组方便操作
        print(f"SHAP 值数组形状: {shap_values_np.shape}") # 应该是 (num_classes, num_samples, C, H, W)
        print(f"所有 SHAP 值的最小值: {shap_values_np.min()}")
        print(f"所有 SHAP 值的最大值: {shap_values_np.max()}")
        print(f"所有 SHAP 值的平均绝对值: {np.mean(np.abs(shap_values_np))}")
        # --- 检查结束 ---

        # 5. 可视化 SHAP 结果
        # shap_values 对于多分类任务，是一个列表，每个元素对应一个类的 SHAP 值 (NumPy array)
        # 形状通常是 [num_classes, num_samples, C, H, W]
        # image_plot 需要 NumPy array，格式为 [N, H, W, C] 或 [N, C, H, W]
        # 我们需要将 SHAP 值和图像数据移回 CPU 并转换为 NumPy

        images_np = explain_batch.cpu().numpy().transpose(0, 2, 3, 1) # NCHW -> NHWC

        # 反归一化图像以便更好地可视化
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        images_np_denorm = std * images_np + mean
        images_np_denorm = np.clip(images_np_denorm, 0, 1)

        # 准备标签用于绘图
        class_names=["健康", "建议观察", "建议手术"]
        plot_labels = np.array([class_names] * num_explain) 
            
        # 获取每个样本的真实和预测标签字符串，用于单独显示或在图外显示
        true_labels = [info['true_label'] for info in explain_info]
        predicted_labels = [info['predicted_label'] for info in explain_info]
        sample_titles = [f"样本 {idx+1} (文件: {os.path.basename(explain_info[idx]['image_path'])})\n真实: {class_names[tl]}, 预测: {class_names[pl]}" 
                        for idx, (tl, pl) in enumerate(zip(true_labels, predicted_labels))]

        # 调整 shap_values 格式以匹配 image_plot 的期望
        shap_values_np = [sv.transpose(0, 2, 3, 1) for sv in shap_values] # NCHW -> NHWC for each class
        shap_values_plot = np.stack(shap_values_np, axis=-1) # Stack along the last axis -> [N, H, W, C, num_classes]

        print("生成 SHAP 图像...")
        # --- (可选) 在调用 image_plot 之前单独显示每个样本及其 True/Pred 标签 ---
        #for i in range(num_explain):
        #    plt.figure()
        #    plt.imshow(images_np_denorm[i])
        #    plt.title(sample_titles[i])
        #    plt.axis('off')
        #    plt.show()

        #shap.image_plot(shap_values_plot * shap_scaling_factor,
        #                images_np_denorm, 
        #                labels=plot_labels, # 为每个类别提供标签
        #                true_labels=None, # true_labels 已包含在 labels 中
        #                show=True) # 直接显示图像

        for i in range(num_explain):
            # 获取第 i 个样本的 SHAP 值 (对所有类别)
            # 确保 SHAP 值和图像数据都保持批次维度为 1
            # explainer.shap_values 返回的是 list of (N, C, H, W) or (N, H, W, C)
            # 我们需要将其处理成 list of (1, H, W, C)
            shap_values_for_sample_i_list = []
            for sv_per_class in shap_values: # Loop through classes
                # sv_per_class shape is (num_samples, C, H, W) or (num_samples, H, W, C)
                # Extract the i-th sample, keeping batch dim: (1, C, H, W) or (1, H, W, C)
                sv_sample_i = sv_per_class[i:i+1] 
                
                # Ensure it's in NHWC format (Batch, Height, Width, Channel) for image_plot if needed
                # Assuming sv_per_class was (N, C, H, W) as is common for PyTorch outputs
                if sv_sample_i.shape[1] == images_np_denorm.shape[-1] and sv_sample_i.shape[2:] == images_np_denorm.shape[1:3]: # Simple check for NCHW
                     # Transpose from (1, C, H, W) to (1, H, W, C)
                    sv_sample_i = sv_sample_i.transpose(0, 2, 3, 1) 
                # else: assume it's already (1, H, W, C) or (1, H, W)

                shap_values_for_sample_i_list.append(sv_sample_i)


            # 获取第 i 个样本的图像数据 (保持批次维度为 1)
            image_for_sample_i_batch = images_np_denorm[i:i+1] # (1, H, W, C)

            # 调用 shap.image_plot 绘制当前样本的解释
            # pixel_values 需要 (N, H, W, C) -> image_for_sample_i_batch is (1, H, W, C)
            # shap_values 需要 list of (N, C, H, W) or list of (N, H, W, C) -> shap_values_for_sample_i_list is list of (1, H, W, C)
            # labels 需要 list of strings (类别名称) -> class_names is list of strings
            shap.image_plot(
                [sv * shap_scaling_factor for sv in shap_values_for_sample_i_list], # Apply scaling to SHAP values, pass list of (1, H, W, C)
                image_for_sample_i_batch, # (1, H, W, C)
                labels=class_names, # Use class names for columns
                show=False # Don't show yet, add title first
            )

            # 获取当前生成的 matplotlib 图和轴
            fig = plt.gcf()
            # 添加总体的标题
            fig.suptitle(sample_titles[i], fontsize=12, y=1.02) # y 调整标题位置，避免与子图重叠

            # 现在显示带有标题的图
            plt.show()
else:
    print("\n没有错误分类的样本可供 SHAP 解释。")

# 模型保存和加载

In [ ]:
# 保存模型参数
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"模型参数已保存到 {path}")

# 加载模型参数
def load_model(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device, weights_only=True))
    model = model.to(device)
    print(f"模型参数已从 {path} 加载")
    return model

# 保存训练好的模型
model_save_path = f"./{train_model.__name__}_pterygium_classifier_base.pth"
save_model(model, model_save_path)

# 模型测试和预测
使用训练好的模型对新图像进行预测，并展示几个预测示例。

In [ ]:
# 模型测试和预测
import shutil


def predict_image(model, image_path, transform, device):
    """
    使用训练好的模型对单张图像进行预测
    :param model: 训练好的模型
    :param image_path: 图像路径
    :param transform: 图像预处理变换
    :param device: 设备（CPU 或 GPU）
    :return: 预测类别
    """
    model.eval()  # 设置模型为评估模式
    image = Image.open(image_path).convert("RGB")  # 加载图像并转换为RGB
    image = transform(image).unsqueeze(0).to(device)  # 应用预处理并添加批次维度

    with torch.no_grad():
        outputs = model(image)  # 前向传播
        _, predicted = outputs.max(1)  # 获取预测类别
    return predicted.item()

def test_model_on_val(model, device, input_dir, temp_dir, transform):
    os.makedirs(temp_dir, exist_ok=True)
    image_paths = glob.glob(os.path.join(input_dir, "*.png"))
    # 删除临时目录中的旧文件
    for img_path in glob.glob(os.path.join(temp_dir, "*.png")):
        os.remove(img_path)
    results = []
    
    for img_path in tqdm(image_paths, desc="Processing images", leave=True):
        base_name = os.path.splitext(os.path.basename(img_path))[0]  # ...获取文件名（不带扩展）...
        output_path = os.path.join(temp_dir, f"{base_name}.{output_format.lower()}")
        if not os.path.exists(output_path):
            try:
                if resize_and_save_image({"Image": base_name}, input_dir, temp_dir, resize_transform, device):
                    predicted_class = predict_image(model, output_path, transform, device)
                    results.append({"Image": base_name, "Pterygium": predicted_class})
                else:
                    tqdm.write(f"Resize失败: {img_path}")
            except Exception as e:
                tqdm.write(f"处理图像 {img_path} 时出错: {e}")
    
    results.sort(key=lambda x: int(x["Image"]))
    df = pd.DataFrame(results, columns=["Image", "Pterygium"])
    result_path = os.path.join(kaggle_temp_path, "Classification_Results.xlsx")
    df.to_excel(result_path, index=False)
    print(f"分类结果已保存到 {result_path}")
    print(f'删除验证集缓存数据 {temp_dir}')
    for img_path in glob.glob(os.path.join(temp_dir, "*.png")):
        os.remove(img_path)

# 加载保存的模型并进行推理
loaded_model = train_model(num_classes=3)
loaded_model = load_model(loaded_model, model_save_path, device)

In [ ]:
try:
    # 单文件推理
    test_image_path = "./test_images/sample_image.png"  # 替换为实际测试图像路径
    predicted_class = predict_image(loaded_model, test_image_path, val_transform, device)
    class_names = ["健康", "建议观察", "建议手术"]
    print(f"加载模型后预测结果: 图像 {os.path.basename(test_image_path)}, 预测类别: {class_names[predicted_class]}")
except:
    pass

In [ ]:
# 验证集推理
val_temp_dir = os.path.join(kaggle_temp_path, "val_resized")
test_model_on_val(loaded_model, device, val_image_dir, val_temp_dir, val_transform)

# 基线模型波动范围检测
K 折交叉验证 (K=5)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset

# --- 交叉验证参数 ---
K = 5  # 折数
base_seed = 420 # 用于 KFold 分割的随机种子

# --- 存储结果 ---
fold_final_val_accuracies = []
fold_final_macro_f1_scores = []
fold_learning_curves = []
# fold_best_model_state_dicts = [] # 如果需要保存每折的最佳模型权重

# --- 准备完整数据集 (加载一次，稍后根据需要应用变换) ---
try:
    full_labels_df = pd.read_excel(label_file)
    # 获取用于分层的标签数组
    all_original_labels = full_labels_df['Pterygium'].values
    # 创建基础数据集实例 (稍后在创建 Subset 时应用变换)
    # 需要两个实例：一个应用训练变换，一个应用验证变换
    base_train_dataset = PterygiumDataset(label_file=label_file, image_dir=image_dir, transform=train_transform)
    base_val_dataset = PterygiumDataset(label_file=label_file, image_dir=image_dir, transform=val_transform)
    # 确保数据集大小和标签数量匹配
    assert len(base_train_dataset) == len(all_original_labels), "Dataset size and label count mismatch!"
except Exception as e:
    print(f"Error preparing full dataset for K-Fold: {e}")
    # 如果无法准备数据，则跳过交叉验证
    # (或者根据需要添加退出逻辑)
    K = 0 # 设置 K 为 0 以跳过循环

# --- 定义损失函数 (可以在循环外定义，因为无状态) ---
base_loss_fn = nn.CrossEntropyLoss()
criterion = HighlightRegularizedLoss(base_loss_fn=base_loss_fn, lambda_reg=0.01) # 正则项仍未启用

# --- 创建 KFold 分割器 ---
if K > 0:
    skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=base_seed)
    print(f"开始进行 {K}-Fold Cross-Validation...")
else:
    print("Skipping K-Fold Cross-Validation due to data preparation error or K=0.")

# --- K-Fold 循环 ---
current_fold_num = 0 # 手动跟踪折数，因为 enumerate(skf.split(...)) 的第一个元素是 fold index
for train_idx, val_idx in skf.split(np.arange(len(base_train_dataset)), all_original_labels):
    current_fold_num += 1
    fold_id_str = f"Fold {current_fold_num}/{K}"
    print(f"\n--- 开始 {fold_id_str} ---")

    # --- 1. 创建当前折的 Subsets ---
    # 使用带有相应变换的基础数据集创建 Subset
    train_subset = Subset(base_train_dataset, train_idx)
    val_subset = Subset(base_val_dataset, val_idx)
    print(f"Fold {current_fold_num}: Train size={len(train_subset)}, Val size={len(val_subset)}")

    # --- 2. 创建当前折的 DataLoaders ---
    train_loader_fold = DataLoader(train_subset,
                                batch_size=64,
                                shuffle=True, # 训练集需要 shuffle
                                num_workers=num_workers,
                                prefetch_factor=2 if platform.system() == "Windows" else 10,
                                pin_memory=False if platform.system() == "Windows" else True)
    val_loader_fold = DataLoader(val_subset,
                                batch_size=64,
                                shuffle=False, # 验证集不需要 shuffle
                                num_workers=num_workers,
                                prefetch_factor=2 if platform.system() == "Windows" else 10,
                                pin_memory=False if platform.system() == "Windows" else True)

    # --- 3. 为当前折创建新的模型、优化器和调度器实例 ---
    model_fold = train_model(num_classes=3).to(device) # 使用与之前相同的模型
    optimizer_fold = optim.AdamW(model_fold.parameters(), lr=train_lr, weight_decay=train_weight_decay) # 使用基线超参数
    scheduler_fold = optim.lr_scheduler.CosineAnnealingLR(optimizer_fold, T_max=num_epochs, eta_min=1e-6) # T_max 应为总 epochs

    # --- 4. 调用训练和验证函数 ---
    final_acc, final_f1, history, best_state = train_validate_model(
        model=model_fold,
        train_loader=train_loader_fold,
        val_loader=val_loader_fold,
        criterion=criterion,
        optimizer=optimizer_fold,
        scheduler=scheduler_fold,
        num_epochs=num_epochs, # 使用之前定义的 num_epochs
        device=device,
        run_identifier=fold_id_str # 传递折信息给日志
    )

    # --- 5. 存储当前折的结果 ---
    fold_final_val_accuracies.append(final_acc)
    fold_final_macro_f1_scores.append(final_f1)
    fold_learning_curves.append(history)
    # if best_state: # 如果需要保存每折的最佳模型权重
    #     fold_best_model_state_dicts.append(best_state)

# --- 6. K-Fold 循环结束后，进行分析和可视化 ---
if K > 0 and fold_final_val_accuracies: # 确保有结果可分析
    print("\n--- K-Fold Cross-Validation 结果分析 ---")

    # 计算平均值和标准差
    mean_accuracy = np.mean(fold_final_val_accuracies)
    std_accuracy = np.std(fold_final_val_accuracies)
    mean_f1 = np.mean(fold_final_macro_f1_scores)
    std_f1 = np.std(fold_final_macro_f1_scores)

    print(f"平均验证准确率 (across {K} folds): {mean_accuracy:.4f}%")
    print(f"验证准确率标准差 (across {K} folds): {std_accuracy:.4f}%")
    print(f"平均验证Macro F1 (across {K} folds): {mean_f1:.4f}")
    print(f"验证Macro F1标准差 (across {K} folds): {std_f1:.4f}")

    print("\n每折的最终验证准确率:")
    for i, acc in enumerate(fold_final_val_accuracies):
        print(f"  Fold {i+1}: {acc:.4f}%")

    print("\n每折的最终验证Macro F1:")
    for i, f1 in enumerate(fold_final_macro_f1_scores):
        print(f"  Fold {i+1}: {f1:.4f}")

    # --- 7. 可视化学习曲线 ---
    plt.figure(figsize=(12, 8))
    for i, curve in enumerate(fold_learning_curves):
        epochs_run = range(1, len(curve) + 1)
        plt.plot(epochs_run, curve, label=f'Fold {i+1}', alpha=0.7)

    plt.title(f'{K}-Fold Cross-Validation: 验证准确率学习曲线')
    plt.xlabel('Epoch')
    plt.ylabel('验证准确率 (%)')
    plt.legend(loc='lower right', ncol=min(K, 5)) # 控制图例列数
    plt.grid(True)
    # 动态调整Y轴范围
    min_y_val = np.min([min(c) for c in fold_learning_curves if c]) if any(fold_learning_curves) else 80
    max_y_val = np.max([max(c) for c in fold_learning_curves if c]) if any(fold_learning_curves) else 100
    plt.ylim(bottom=max(0, min_y_val - 2), top=min(100, max_y_val + 2))
    plt.show()

elif K > 0:
    print("\nK-Fold Cross-Validation ran, but no results were collected (possibly due to errors in training).")